In [1]:
!pip install --upgrade gradio

   ---------------------------------------- 0.0/24.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/24.2 MB 2.1 MB/s eta 0:00:12
   -- ------------------------------------- 1.6/24.2 MB 2.8 MB/s eta 0:00:09
   ---- ----------------------------------- 2.6/24.2 MB 3.7 MB/s eta 0:00:06
   ---- ----------------------------------- 2.6/24.2 MB 3.7 MB/s eta 0:00:06
   ----- ---------------------------------- 3.4/24.2 MB 3.3 MB/s eta 0:00:07
   ------ --------------------------------- 4.2/24.2 MB 3.3 MB/s eta 0:00:07
   ------ --------------------------------- 4.2/24.2 MB 3.3 MB/s eta 0:00:07
   -------- ------------------------------- 5.2/24.2 MB 3.0 MB/s eta 0:00:07
   --------- ------------------------------ 6.0/24.2 MB 3.0 MB/s eta 0:00:06
   ---------- ----------------------------- 6.3/24.2 MB 2.9 MB/s eta 0:00:07
   ----------- ---------------------------- 6.8/24.2 MB 2.8 MB/s eta 0:00:07
   ----------

In [2]:
import gradio as gr

In [3]:
def shout(text):
    print("Shout has been called with ",text)
    return text.upper()

In [4]:
shout("Liza")

Shout has been called with  Liza


'LIZA'

In [8]:
gr.Interface(fn=shout, inputs = "textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [10]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(auth=("liz", "math"))

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


In [11]:
# Adding a little more:

message_input = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=shout,
    title="Shout", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


Shout has been called with  howdy


In [15]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [17]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [18]:
openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)

In [19]:
system_message = "You are a helpful assistant"

def message_gpt(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content
message_gpt("How do you do?")

"Hello! I'm doing well, thank you. How can I assist you today?"

In [20]:
message_input = gr.Textbox(label="Your message:", info="Enter a message", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=message_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


In [23]:
system_message = "You are a helpful assistant that returns output in markdown with code blocks"

message_input = gr.Textbox(label="Your message:", info="Enter a message", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
             ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


# Lets stream

In [24]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [25]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


In [26]:
def stream_claude(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = anthropic.chat.completions.create(
        model='claude-sonnet-4-5-20250929',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [28]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [29]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="LLMs", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Explain the Transformer architecture to a layperson", "GPT"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "Claude"]
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.
